In [ ]:
import re
import torch
from transformers import BertTokenizer, BertModel, pipeline
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load Pretrained BERT model for semantic job matching
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
bert_model = BertModel.from_pretrained("bert-base-cased")

# Load Generative AI model for job recommendation
gpt_model = pipeline("text-generation", model="gpt2")

# Sample resume text
resume_text = """
John Doe
Email: john.doe@example.com
Phone: +1 234 567 890
Skills: Python, Machine Learning, NLP, Deep Learning, Data Analysis
Experience:
  - Data Scientist at XYZ Corp (2020-2024)
  - AI Engineer at ABC Tech (2018-2020)
Education: MSc in Computer Science
"""

# Sample job descriptions
job_descriptions = [
    "Looking for a Data Scientist with expertise in Machine Learning, NLP, and Python.",
    "Seeking a Software Engineer with Java and backend development experience.",
    "Hiring an AI Engineer skilled in Deep Learning, Computer Vision, and TensorFlow."
]

# Function to extract skills from resume
def extract_skills(text):
    skills_pattern = re.compile(r"Skills:\s*(.*)")
    match = skills_pattern.search(text)
    return match.group(1).split(", ") if match else []

# Convert text to BERT embeddings
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# Match resume with job descriptions
resume_embedding = get_bert_embedding(resume_text)
job_embeddings = [get_bert_embedding(job) for job in job_descriptions]
similarity_scores = [cosine_similarity(resume_embedding, job_emb)[0][0] for job_emb in job_embeddings]

# Get best match
best_match_idx = np.argmax(similarity_scores)
best_job = job_descriptions[best_match_idx]

# Generate AI-based job recommendation
prompt = f"Candidate skills: {extract_skills(resume_text)}. Best job match: {best_job}. Explain why this job is a good fit."
job_recommendation = gpt_model(prompt, max_length=100, num_return_sequences=1)[0]["generated_text"]

# Display results
print("🔹 **Top Job Match:**", best_job)
print("\n💡 **AI-Generated Job Recommendation:**\n", job_recommendation)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


🔹 **Top Job Match:** Looking for a Data Scientist with expertise in Machine Learning, NLP, and Python.

💡 **AI-Generated Job Recommendation:**
 Candidate skills: ['Python', 'Machine Learning', 'NLP', 'Deep Learning', 'Data Analysis']. Best job match: Looking for a Data Scientist with expertise in Machine Learning, NLP, and Python.. Explain why this job is a good fit. Must be able to communicate concisely and clearly about the current situation, its benefits and challenges, what you will be doing, what your position will be, and who your employer is.

About the Author Ryan Ruse is
